In [2]:
import cv2 as cv
import numpy as np
import os

reading images

In [3]:
img = cv.imread('images/car.jpg')
cv.imshow('car',img)
cv.waitKey(0)

-1

reading videos

In [4]:
capture = cv.VideoCapture('videos/vid2.mp4')
while True:
    isTrue, frame = capture.read()
    cv.imshow('video', frame)
    if cv.waitKey(20) & 0xFF==ord('d'):
        break
        
capture.release()
cv.destroyAllWindows()

Rescaling

In [4]:
def rescaleFrame(frame, scale=0.75):
    width = int(frame.shape[1]*scale)
    height = int(frame.shape[0]*scale)
    newdim = (width, height)
    return cv.resize(frame, newdim, interpolation=cv.INTER_AREA)


In [ ]:
#video rescaling
capture = cv.VideoCapture('videos/vid2.mp4')
while True:
    isTrue, frame = capture.read()
    frame_rescaled = rescaleFrame(frame, scale=0.2)
    cv.imshow('video resized', frame_rescaled)
    if cv.waitKey(20) & 0xFF==ord('d'):
        break
        
capture.release()
cv.destroyAllWindows()

In [5]:
#image rescaling
img_rescaled= rescaleFrame(img,0.2)
cv.imshow('car',img_rescaled)
cv.waitKey(0)

-1

In [ ]:
##for live video only....
def changeRes(width, height):
    capture.set(3, width)
    capture.set(4,height)

drawing

In [2]:
blankPic = np.zeros((500,500,3))
cv.imshow('blank', blankPic)
blankPic[:]= 0, 255, 0
cv.imshow('green', blankPic)
cv.waitKey(0)

-1

In [9]:
randpic = (np.random.randint(0,2,500*500*3).reshape(500,500,3))/1.0
cv.imshow('random',randpic)
cv.waitKey(0)

-1

In [20]:
##rectangle
canvas = np.zeros((500,500,3))
cv.rectangle(canvas, (0,0), (250,250), (0,255,0), thickness=2)
cv.imshow('rect',canvas)
cv.waitKey(0)

-1

In [13]:
cv.rectangle(canvas, (0,0), (250,250), (0,255,0), thickness=-1) ## fill rectangle
cv.imshow('rect',canvas)
cv.waitKey(0)

-1

In [14]:
##circle
cv.circle(canvas,(250,250), 125, (0,0,255), thickness=1)
cv.imshow('circle', canvas)
cv.waitKey(0)

-1

In [16]:
##line
cv.line(canvas, (250,250), (500,500), (255,0,0), thickness=1)
cv.imshow('line', canvas)
cv.waitKey(0)

-1

In [21]:
##text
cv.putText(canvas, 'Hello', (255,255), cv.FONT_HERSHEY_TRIPLEX, 0.5, (0,255,255), thickness=1)
cv.imshow('text', canvas)
cv.waitKey(0)

-1

basic functionalities

In [6]:
# converting to grayscale
img_BnW = cv.cvtColor(img_rescaled, cv.COLOR_BGR2GRAY)
cv.imshow('carB&W', img_BnW)
cv.waitKey(0)

-1

In [19]:
# blurring
img_blur = cv.GaussianBlur(img_rescaled, (3,3), cv.BORDER_DEFAULT)
cv.imshow('imgBlurred', img_blur)
cv.waitKey(0)

img_blur2 = cv.GaussianBlur(img_rescaled, (7,7), cv.BORDER_DEFAULT)
cv.imshow('imgBlurred', img_blur2)
cv.waitKey(0)

-1

In [7]:
# Edge Cascading
img_edges = cv.Canny(img_rescaled, 125,175)
cv.imshow('canny edges', img_edges)
cv.waitKey(0)

img_edges2 = cv.Canny(img_blur, 125,175)
cv.imshow('canny edges', img_edges2)
cv.waitKey(0)

NameError: name 'img_blur' is not defined

In [35]:
# dilating
img_dilated = cv.dilate(img_rescaled, (7,7), iterations=3)
cv.imshow('dilated', img_dilated)
cv.waitKey(0)

img_dilated2 = cv.dilate(img_edges, (7,7), iterations=3)
cv.imshow('dilated', img_dilated2)
cv.waitKey(0)

-1

In [36]:
# eroding
img_eroded = cv.erode(img_rescaled, (7,7), iterations=3)
cv.imshow('eroded', img_eroded)
cv.waitKey(0)

img_eroded = cv.erode(img_dilated2, (7,7), iterations=3)
cv.imshow('eroded', img_eroded)
cv.waitKey(0)

# * img -> cascaded_Img -> dilated -> eroded -> cascaded_Img

-1

In [38]:
# resizing
img_resized = cv.resize(img, (250,250), interpolation=cv.INTER_AREA) ##cv.INTER_LINEAR,<cv.INTER_CUBIC
cv.imshow('resized', img_resized)
cv.waitKey(0)

-1

In [40]:
# cropping
img_cropped = img_rescaled[0:250,250:500]  ## y1,y2,x1,x2
cv.imshow('cropped', img_cropped)
cv.waitKey(0)

-1

Transformation

In [41]:
# translation
def translate(img, x, y):
    transMat = np.float32([[1,0,x], [0,1,y]])
    dims = (img.shape[1], img.shape[0])
    return cv.warpAffine(img, transMat, dims)

img_translated = translate(img_rescaled, 100, 100)
cv.imshow('translated', img_translated)
cv.waitKey(0)

-1

In [45]:
# rotation
def rotate(img, angle, rotPoint=None):
    (height, width) = img.shape[:2]
    if rotPoint is None:
        rotPoint = (width//2, height//2)
    rotMat = cv.getRotationMatrix2D(rotPoint, angle, 1.0)
    dims = (width, height)
    return cv.warpAffine(img, rotMat, dims)

img_rotated = rotate(img_rescaled, 45)
cv.imshow('rotated', img_rotated)
cv.waitKey(0)

img_rotated = rotate(img_rotated, -45)
cv.imshow('rotated', img_rotated)
cv.waitKey(0)

-1

In [50]:
# flipping
img_flipped = cv.flip(img_rescaled, 1) ## 0 Vf, 1 Hf, -1 VHf
cv.imshow('flipped', img_flipped)
cv.waitKey(0)

-1

Contour Detection

In [9]:
cv.imshow('BnW', img_BnW)
cv.imshow('edges', img_edges)
cv.waitKey(0)

# contours, hierarchies = cv.findContours(img_edges, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
# print(f'{len(contours)} Contours Found')  #-- 846

ret, img_thresh = cv.threshold(img_BnW, 125, 255, cv.THRESH_BINARY)
cv.imshow('Thresh', img_thresh)
cv.waitKey(0)

contours, hierarchies = cv.findContours(img_thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)  # takes only grayscaled images
print(f'{len(contours)} Contours Found') #-- 597


597 Contours Found


In [13]:
# displaying the contours
canvas = np.zeros(img_rescaled.shape)
cv.drawContours(canvas, contours, -1, (0,0,255), thickness=1)
cv.imshow('contours drawn', canvas)
cv.imshow('edges', img_edges)
cv.waitKey(0)


-1